In [1]:
#files
import sys
from os import listdir
sys.path.append('../data')
sys.path.append('../Librerias')
from os.path import isfile, join


#own libs
import utils
import caract as dc
import dataset as ds


#data and visualization
import copy
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

sns.set_style("darkgrid")

c:\Users\consu\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [45]:
217+25

242

In [3]:
#fpath = "/Users/granosftp/Documents/GitHub/Tesis/data/high/"
fpath =  "/Users/consu/OneDrive/Documentos/GitHub/Tesis/data/low/"
onlyfiles = [f for f in listdir(fpath) if isfile(join(fpath, f))]

cutoff = [8/1000, 11/1000]

#load data
frames = []
for fname in onlyfiles:
    if fname.endswith(".mat"):
        set =  ds.MatFileToDataFrame(fpath, fname)
        df = set.get_dataframe(cutoff)
        frames.append(df)
        print(fname)

frames = frames[:2]

defectos_frames = []
taus = []
for i, df in enumerate(frames):
    aux = dc.CaractDefect(df)
    defectos_frames.append(aux)
    taus.append(aux.get_tau()[1])
    sys.stdout.write(str(i) + '\r')


hilbert_frames = []
for i, df in enumerate(defectos_frames):
    aux, _ = df.get_hilbert()
    hilbert_frames.append(aux)
    sys.stdout.write(str(i) + '\r')


frames_all = []

for i in range(len(frames)):
    frames_all.append(pd.merge(frames[i], hilbert_frames[i], on='Hilbert Transform', how='outer'))

frames_all[0]

file10_080124.mat
file11_080124.mat
file12_080124.mat
file13_080124.mat
file15_080124.mat
file16_080124.mat
file17_080124.mat
file18_080124.mat
file1_050124.mat
file1_080124.mat
file2_050124.mat
file3_080124.mat
file7_080124.mat
file8_080124.mat
file9_080124.mat
low1_100124.mat
low2_100124.mat
low3_100124.mat
low5_100124.mat


,Original Signal,Signal - Mean,Filtered Signal,Hilbert Transform,Amplitude,Instantaneous Phase,Gradient Phase
0,0.076967,-0.041045,-0.005045,-0.005045+0.003866j,0.006356,2.487745,-0.050221
1,0.076638,-0.041374,-0.004841,-0.004841+0.000279j,0.004849,3.083952,-0.023636
2,0.073022,-0.044991,-0.004633,-0.004633+0.000083j,0.004634,3.123610,0.001438
3,0.073022,-0.044991,-0.004421,-0.004421-0.001203j,0.004582,3.407171,0.023619
4,0.071378,-0.046635,-0.004205,-0.004205-0.001356j,0.004419,3.453607,0.041826
...,...,...,...,...,...,...,...
599995,0.176268,0.058256,0.000015,0.000015+0.000493j,0.000493,30135.697377,-0.289769
599996,0.177254,0.059242,0.000033,0.000033+0.000343j,0.000345,30135.630801,-0.245612
599997,0.179227,0.061215,0.000050,0.000050+0.001284j,0.001285,30135.688260,-0.187389
599998,0.179885,0.061872,0.000066,0.000066+0.001095j,0.001097,30135.667052,-0.118399


In [5]:
serie = frames_all[0]
serie_subsample = serie.iloc[::2]
serie_subsample.head()

serie3_subsample = serie_subsample.iloc[::3]
serie3_subsample[:167]


,Original Signal,Signal - Mean,Filtered Signal,Hilbert Transform,Amplitude,Instantaneous Phase,Gradient Phase
0,0.076967,-0.041045,-0.005045,-0.005045+0.003866j,0.006356,2.487745,-0.050221
6,0.069733,-0.048279,-0.003764,-0.003764-0.002288j,0.004405,3.687723,0.063977
12,0.067103,-0.050909,-0.002374,-0.002374-0.003953j,0.004611,4.171536,0.037612
18,0.070062,-0.047950,-0.000930,-0.000930-0.004792j,0.004882,4.520614,-0.005992
24,0.080584,-0.037428,0.000507,0.000507-0.005115j,0.005140,4.811198,-0.008164
...,...,...,...,...,...,...,...
972,0.162129,0.044117,-0.000020,-0.000020+0.000260j,0.000261,39.344843,-0.039660
978,0.151607,0.033595,-0.000050,-0.000050+0.000290j,0.000294,39.438999,-0.031099
984,0.141743,0.023731,-0.000098,-0.000098+0.000311j,0.000326,39.574395,-0.025385
990,0.137139,0.019127,-0.000160,-0.000160+0.000317j,0.000355,39.737982,-0.021377


In [6]:
from scipy.signal import butter, filtfilt

order = 4
cutoff_freq = 100

filtered2_df = serie_subsample.copy()
filtered3_df = serie3_subsample.copy()

for column in filtered2_df.columns:
    b, a = butter(order, cutoff_freq, fs=1000, btype='low', analog=False, output='ba')
    filtered2_df[column] = filtfilt(b, a, filtered2_df[column])

filtered2_df.head()

for column in filtered3_df.columns:
    b, a = butter(order, cutoff_freq, fs=1000, btype='low', analog=False, output='ba')
    filtered3_df[column] = filtfilt(b, a, filtered3_df[column])



In [7]:
fig = go.Figure()

# Add trace for frames_all[0]
fig.add_trace(go.Scatter(x=frames_all[0].index[:1000], y=frames_all[0]['Amplitude'][:1000],
                         mode='lines', name='frames_all[0]'))

# Add trace for filtered_df
fig.add_trace(go.Scatter(x=filtered2_df.index[:500], y=filtered2_df['Amplitude'][:500],
                         mode='lines', name='subsampleo de 1/2'))

# Add trace for filtered_df
fig.add_trace(go.Scatter(x=filtered3_df.index[:167], y=filtered3_df['Amplitude'][:167],
                         mode='lines', name='subsampleo de 1/3'))

# Update layout
fig.update_layout(title='Amplitude Comparison',
                  xaxis_title='Index',
                  yaxis_title='Amplitude')

# Show the figure
fig.show()


In [8]:
fig = go.Figure()

# Add trace for frames_all[0]
fig.add_trace(go.Scatter(x=frames_all[0].index[:1000], y=frames_all[0]['Amplitude'][:1000],
                         mode='lines+markers', name='frames_all[0]'))

# Add trace for filtered_df
fig.add_trace(go.Scatter(x=filtered2_df.index[:500], y=filtered2_df['Amplitude'][:500],
                         mode='markers', name='subsampleo de 1/2'))

# Add trace for filtered_df
fig.add_trace(go.Scatter(x=filtered3_df.index[:167], y=filtered3_df['Amplitude'][:167],
                         mode='lines+markers', name='subsampleo de 1/3'))

# Update layout
fig.update_layout(title='Amplitude Comparison',
                  xaxis_title='Index',
                  yaxis_title='Amplitude')

# Show the figure
fig.show()

## **welch para las amplitudes**

así me aseguro de que no hay frecuencias fantasmas y de todas tienen la misma psd lo que las hace equivalente para su uso posterior

In [9]:
from scipy.signal import welch

#Calculate the Welch PSD for frames_all[0]
freq_all, psd_all = welch(frames_all[0]['Amplitude'], fs=1000, nperseg=2000, scaling = 'spectrum', window='hamming')


# Calculate the Welch PSD for filtered_df2
freq2, psd2 = welch(filtered2_df['Amplitude'], fs=500, nperseg=2000, scaling = 'spectrum', window='hamming')

# Calculate the Welch PSD for filtered_df3
freq3, psd3 = welch(filtered3_df['Amplitude'], fs=167, nperseg=2000, scaling = 'spectrum', window='hamming')

fig = go.Figure()

# Add trace for frames_all[0]
fig.add_trace(go.Scatter(x=freq_all, y=psd_all,
                         mode='lines', name='frames_all[0]'))

# Add trace for filtered_df2
fig.add_trace(go.Scatter(x=freq2, y=psd2,
                         mode='lines', name='filtered_df2'))

# Add trace for filtered_df3
fig.add_trace(go.Scatter(x=freq3, y=psd3,
                         mode='lines', name='filtered_df3'))

# Update layout
fig.update_layout(title='Power Spectral Density of Amplitude',
                  xaxis_title='Frequency',
                  yaxis_title='PSD',
                  xaxis_type='log',
                  yaxis_type='log')


# Show the figure
fig.show()



In [10]:
#Calculate the Welch PSD for frames_all[0]
freq_all, psd_all = welch(frames_all[0]['Filtered Signal'], fs=1000, nperseg=2000, scaling = 'spectrum', window='hamming')


# Calculate the Welch PSD for filtered_df2
freq2, psd2 = welch(filtered2_df['Filtered Signal'], fs=500, nperseg=2000, scaling = 'spectrum', window='hamming')

# Calculate the Welch PSD for filtered_df3
freq3, psd3 = welch(filtered3_df['Filtered Signal'], fs=167, nperseg=2000, scaling = 'spectrum', window='hamming')

fig = go.Figure()

# Add trace for frames_all[0]
fig.add_trace(go.Scatter(x=freq_all, y=psd_all,
                         mode='lines', name='frames_all[0]'))

# Add trace for filtered_df2
fig.add_trace(go.Scatter(x=freq2, y=psd2,
                         mode='lines', name='filtered_df2'))

# Add trace for filtered_df3
fig.add_trace(go.Scatter(x=freq3, y=psd3,
                         mode='lines', name='filtered_df3'))

# Update layout
fig.update_layout(title='Power Spectral Density of Filtered Signal',
                  xaxis_title='Frequency',
                  yaxis_title='PSD',
                  xaxis_type='log',
                  yaxis_type='log')


# Show the figure
fig.show()


### **conclusiones**

* Se puede hacer subsampleo a 500Hz,no se pierde mucha información tanto en la curva original como ne la amplitud. Subsampleo mayor a eso, se generan muchas perdidas. Eso se debe a que los defectos son muy rápidos, por lo que el muestro podría, perfectamente eliminarlos de la señal.